# Explore Pipeline

We wish to explore the impact of our data pipeline; we will consider six things:

- train-validation split size.
- Model batch size.
- rescaling
- including rotations to the images.
- including translations of the images.
- including added noise to the brightness of the images.
- zooming on the images.

## Summary

Based on the below findings; it seems like data augmentation of the following.

|Augmentation|Value|
|------------|-----|
|train-val split|0.8 / 0.9|
|batch size|64 or 128|
|rescaling| 1/255.|
|rotations| 0.05 (18deg)|
|translations| 0.1|
|brightness| 0.1|
|zoom|0.1|

In [16]:
import os
import sys
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src import DrDataLoader
import mlflow
import mlflow.tensorflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.tensorflow.autolog()
load_dotenv(find_dotenv())

data_loader = DrDataLoader(Path('../') / Path(os.getenv('RAW_FP'))) #type: ignore

def get_rescale_im(factor):
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(factor)
    ])

def get_rotate_im(rotation):
    return tf.keras.Sequential([
        tf.keras.layers.RandomRotation(factor=rotation)
    ])

def get_translate_im(translation):
    return tf.keras.Sequential([
        tf.keras.layers.RandomTranslation(
            height_factor=translation[0],
            width_factor=translation[0]
        )
    ])

def get_brighten_im(brightness):
    return tf.keras.Sequential([
        tf.keras.layers.RandomBrightness(
            factor = brightness
        )
    ])

def get_zoom_im(factor):
    return tf.keras.Sequential([
        tf.keras.layers.RandomZoom(factor)
    ])

AUTOTUNE = tf.data.AUTOTUNE

def prepare(
    ds,
    rescale=None,
    batch_size = None,
    rotation=None,
    translation=None,
    brightness=None,
    zoom=None,
    ):

    if brightness is not None:
        brighten_im = get_brighten_im(brightness)
        ds = ds.map(lambda x, y: (brighten_im(x), y), num_parallel_calls=AUTOTUNE)

    if rescale is not None:
        rescale_im = get_rescale_im(rescale)
        ds = ds.map(lambda x, y: (rescale_im(x), y), num_parallel_calls=AUTOTUNE)
    
    if rotation is not None:
        rotate_im = get_rotate_im(rotation)
        ds = ds.map(lambda x, y: (rotate_im(x), y), num_parallel_calls=AUTOTUNE)
    
    if translation is not None:
        translate_im = get_translate_im(translation)
        ds = ds.map(lambda x, y: (translate_im(x), y), num_parallel_calls=AUTOTUNE)

    if zoom is not None:
        zoom_im = get_zoom_im(zoom)
        ds = ds.map(lambda x, y: (zoom_im(x), y), num_parallel_calls=AUTOTUNE)

    if batch_size is not None:
        ds = ds.batch(batch_size)

    return ds


2022/12/02 16:32:54 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


## Experiment 1
We explore the impact of the train - validation split size.

In [6]:
experiment_id = mlflow.create_experiment(
    'pipeline-experiments-split_size', #type: ignore
    artifact_location=Path.cwd().joinpath('../artifacts').as_uri(),
    tags={'version': 'v1', 'priority': 'P1'},
)
experiment = mlflow.get_experiment(experiment_id)
print(f'Name: {experiment.name}')
print(f'Experiment_id: {experiment.experiment_id}')
print(f'Artifact Location: {experiment.artifact_location}')
print(f'Tags: {experiment.tags}')
print(f'Lifecycle_stage: {experiment.lifecycle_stage}')
print(f'Creation timestamp: {experiment.creation_time}')

BATCH_SIZE = 128
SPLIT_SIZES = [0.5, 0.6, 0.7, 0.8, 0.9]
RESCALE = None
ROTATION = None
TRANSLATION = None
BRIGHTEN = None
EPOCHS = 10


for SPLIT_SIZE in SPLIT_SIZES:
    with mlflow.start_run(
        run_name=f'run_split_size_{str(SPLIT_SIZE).zfill(3)}',
        experiment_id=experiment_id
        ):
        mlflow.log_param('SPLIT_SIZE', SPLIT_SIZE)
        mlflow.log_param("BATCH_SIZE", BATCH_SIZE)
        mlflow.log_param('RESCALE', RESCALE)
        mlflow.log_param('ROTATION', ROTATION)
        mlflow.log_param('TRANSLATION', TRANSLATION)
        mlflow.log_param('BRIGHTEN', BRIGHTEN)
        mlflow.log_param('EPOCHS', EPOCHS)


        train_ds, val_ds = tf.keras.utils.split_dataset(
            data_loader.load_training_data(),
            left_size = SPLIT_SIZE,
            shuffle=True
        )

        train_ds = prepare(
            train_ds,
            rescale=RESCALE,
            batch_size=BATCH_SIZE,
            rotation=ROTATION,
            translation=TRANSLATION,
            brightness=BRIGHTEN
        )

        val_ds = prepare(
            val_ds,
            rescale=RESCALE,
            batch_size=BATCH_SIZE,
            rotation=ROTATION,
            translation=TRANSLATION,
            brightness=BRIGHTEN
        )
        
        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10)
            ])  
        model.compile(
            optimizer=tf.keras.optimizers.Adam(0.001),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['sparse_categorical_accuracy'])
        history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Name: pipeline-experiments-split_size
Experiment_id: 1
Artifact Location: file:///Users/b7064522/Documents/DataScience/kaggle-digit-recognizer/notebooks/../artifacts
Tags: {'version': 'v1', 'priority': 'P1'}
Lifecycle_stage: active
Creation timestamp: 1669993601097
Epoch 1/10
  1/165 [..............................] - ETA: 3:24 - loss: 137.2285 - sparse_categorical_accuracy: 0.0781WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0040s vs `on_train_batch_end` time: 0.0266s). Check your callbacks.


165/165 [==============================] - 3s 12ms/step - loss: 9.9703 - sparse_categorical_accuracy: 0.8220 - val_loss: 3.3028 - val_sparse_categorical_accuracy: 0.8870
Epoch 2/10
165/165 [==============================] - 1s 9ms/step - loss: 2.1082 - sparse_categorical_accuracy: 0.9107 - val_loss: 2.1925 - val_sparse_categorical_accuracy: 0.9078
Epoch 3/10
165/165 [==============================] - 1s 8ms/step - loss: 1.0859 - sparse_categorical_accuracy: 0.9340 - val_loss: 1.9465 - val_sparse_categorical_accuracy: 0.9143
Epoch 4/10
165/165 [==============================] - 1s 9ms/step - loss: 0.6912 - sparse_categorical_accuracy: 0.9501 - val_loss: 1.5740 - val_sparse_categorical_accuracy: 0.9249
Epoch 5/10
165/165 [==============================] - 1s 5ms/step - loss: 0.4270 - sparse_categorical_accuracy: 0.9621 - val_loss: 1.5198 - val_sparse_categorical_accuracy: 0.9258
Epoch 6/10
165/165 [==============================] - 1s 4ms/step - loss: 0.3311 - sparse_categorical_accuracy

2022/12/02 15:07:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/b7064522/opt/miniconda3/envs/kdrenv/lib/python3.10/site-packages/mlflow/tensorflow/__init__.py:189: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input."


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmp8605ahti/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmp8605ahti/model/data/model/assets


...

Epoch 1/10
  1/296 [..............................] - ETA: 7:43 - loss: 98.3047 - sparse_categorical_accuracy: 0.2500WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0030s vs `on_train_batch_end` time: 0.0223s). Check your callbacks.


296/296 [==============================] - 3s 6ms/step - loss: 5.2844 - sparse_categorical_accuracy: 0.8500 - val_loss: 1.7116 - val_sparse_categorical_accuracy: 0.8917
Epoch 2/10
296/296 [==============================] - 1s 5ms/step - loss: 0.9131 - sparse_categorical_accuracy: 0.9044 - val_loss: 1.0341 - val_sparse_categorical_accuracy: 0.8938
Epoch 3/10
296/296 [==============================] - 1s 4ms/step - loss: 0.4792 - sparse_categorical_accuracy: 0.9243 - val_loss: 0.6617 - val_sparse_categorical_accuracy: 0.9148
Epoch 4/10
296/296 [==============================] - 1s 4ms/step - loss: 0.2978 - sparse_categorical_accuracy: 0.9434 - val_loss: 0.6133 - val_sparse_categorical_accuracy: 0.9198
Epoch 5/10
296/296 [==============================] - 1s 4ms/step - loss: 0.2107 - sparse_categorical_accuracy: 0.9519 - val_loss: 0.5581 - val_sparse_categorical_accuracy: 0.9243
Epoch 6/10
296/296 [==============================] - 1s 3ms/step - loss: 0.1636 - sparse_categorical_accuracy:

2022/12/02 15:09:39 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/b7064522/opt/miniconda3/envs/kdrenv/lib/python3.10/site-packages/mlflow/tensorflow/__init__.py:189: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input."


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmpm_19_03r/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmpm_19_03r/model/data/model/assets


## Experiment Batch size

We see the impact of batch size.

In [7]:
experiment_id = mlflow.create_experiment(
    'pipeline-experiments-batch_size', #type: ignore
    artifact_location=Path.cwd().joinpath('../artifacts').as_uri(),
    tags={'version': 'v1', 'priority': 'P1'},
)
experiment = mlflow.get_experiment(experiment_id)
print(f'Name: {experiment.name}')
print(f'Experiment_id: {experiment.experiment_id}')
print(f'Artifact Location: {experiment.artifact_location}')
print(f'Tags: {experiment.tags}')
print(f'Lifecycle_stage: {experiment.lifecycle_stage}')
print(f'Creation timestamp: {experiment.creation_time}')

BATCH_SIZES = [32, 64, 128, 256, 512]
SPLIT_SIZE = 0.8
RESCALE = None
ROTATION = None
TRANSLATION = None
BRIGHTEN = None
EPOCHS = 10


for BATCH_SIZE in BATCH_SIZES:
    with mlflow.start_run(
        run_name=f'run_batch_size_{str(BATCH_SIZE).zfill(3)}',
        experiment_id=experiment_id
        ):
        mlflow.log_param('SPLIT_SIZE', SPLIT_SIZE)
        mlflow.log_param("BATCH_SIZE", BATCH_SIZE)
        mlflow.log_param('RESCALE', RESCALE)
        mlflow.log_param('ROTATION', ROTATION)
        mlflow.log_param('TRANSLATION', TRANSLATION)
        mlflow.log_param('BRIGHTEN', BRIGHTEN)
        mlflow.log_param('EPOCHS', EPOCHS)


        train_ds, val_ds = tf.keras.utils.split_dataset(
            data_loader.load_training_data(),
            left_size = SPLIT_SIZE,
            shuffle=True
        )

        train_ds = prepare(
            train_ds,
            rescale=RESCALE,
            batch_size=BATCH_SIZE,
            rotation=ROTATION,
            translation=TRANSLATION,
            brightness=BRIGHTEN
        )

        val_ds = prepare(
            val_ds,
            rescale=RESCALE,
            batch_size=BATCH_SIZE,
            rotation=ROTATION,
            translation=TRANSLATION,
            brightness=BRIGHTEN
        )
        
        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10)
            ])  
        model.compile(
            optimizer=tf.keras.optimizers.Adam(0.001),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['sparse_categorical_accuracy'])
        history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)


Name: pipeline-experiments-batch_size
Experiment_id: 2
Artifact Location: file:///Users/b7064522/Documents/DataScience/kaggle-digit-recognizer/notebooks/../artifacts
Tags: {'version': 'v1', 'priority': 'P1'}
Lifecycle_stage: active
Creation timestamp: 1669993788865
Epoch 1/10
   1/1050 [..............................] - ETA: 17:47 - loss: 143.8230 - sparse_categorical_accuracy: 0.0938WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0029s vs `on_train_batch_end` time: 0.0207s). Check your callbacks.


1050/1050 [==============================] - 6s 4ms/step - loss: 4.1288 - sparse_categorical_accuracy: 0.8606 - val_loss: 0.7886 - val_sparse_categorical_accuracy: 0.8838
Epoch 2/10
1050/1050 [==============================] - 3s 3ms/step - loss: 0.5376 - sparse_categorical_accuracy: 0.8947 - val_loss: 0.5084 - val_sparse_categorical_accuracy: 0.9071
Epoch 3/10
1050/1050 [==============================] - 3s 2ms/step - loss: 0.3566 - sparse_categorical_accuracy: 0.9154 - val_loss: 0.4103 - val_sparse_categorical_accuracy: 0.9183
Epoch 4/10
1050/1050 [==============================] - 3s 2ms/step - loss: 0.3003 - sparse_categorical_accuracy: 0.9270 - val_loss: 0.3792 - val_sparse_categorical_accuracy: 0.9250
Epoch 5/10
1050/1050 [==============================] - 3s 2ms/step - loss: 0.2779 - sparse_categorical_accuracy: 0.9316 - val_loss: 0.4539 - val_sparse_categorical_accuracy: 0.9288
Epoch 6/10
1050/1050 [==============================] - 3s 2ms/step - loss: 0.2651 - sparse_categoric

2022/12/02 15:10:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/b7064522/opt/miniconda3/envs/kdrenv/lib/python3.10/site-packages/mlflow/tensorflow/__init__.py:189: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input."


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmp7sdhmwm1/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmp7sdhmwm1/model/data/model/assets


...

Epoch 1/10
 6/66 [=>............................] - ETA: 0s - loss: 100.0527 - sparse_categorical_accuracy: 0.1908  WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0066s vs `on_train_batch_end` time: 0.0689s). Check your callbacks.


66/66 [==============================] - 4s 20ms/step - loss: 17.8303 - sparse_categorical_accuracy: 0.7488 - val_loss: 4.3591 - val_sparse_categorical_accuracy: 0.8775
Epoch 2/10
66/66 [==============================] - 1s 12ms/step - loss: 3.1410 - sparse_categorical_accuracy: 0.8942 - val_loss: 2.6688 - val_sparse_categorical_accuracy: 0.8998
Epoch 3/10
66/66 [==============================] - 1s 10ms/step - loss: 1.8921 - sparse_categorical_accuracy: 0.9175 - val_loss: 2.1404 - val_sparse_categorical_accuracy: 0.9106
Epoch 4/10
66/66 [==============================] - 1s 10ms/step - loss: 1.2709 - sparse_categorical_accuracy: 0.9345 - val_loss: 1.8445 - val_sparse_categorical_accuracy: 0.9167
Epoch 5/10
66/66 [==============================] - 1s 10ms/step - loss: 0.9187 - sparse_categorical_accuracy: 0.9449 - val_loss: 1.6058 - val_sparse_categorical_accuracy: 0.9260
Epoch 6/10
66/66 [==============================] - 1s 11ms/step - loss: 0.6826 - sparse_categorical_accuracy: 0.95

2022/12/02 15:13:34 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/b7064522/opt/miniconda3/envs/kdrenv/lib/python3.10/site-packages/mlflow/tensorflow/__init__.py:189: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input."


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmp52ow1hnu/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmp52ow1hnu/model/data/model/assets


## Experiment Rescaling images

In [9]:
experiment_id = mlflow.create_experiment(
    'pipeline-experiments-rescale_size', #type: ignore
    artifact_location=Path.cwd().joinpath('../artifacts').as_uri(),
    tags={'version': 'v1', 'priority': 'P1'},
)
experiment = mlflow.get_experiment(experiment_id)
print(f'Name: {experiment.name}')
print(f'Experiment_id: {experiment.experiment_id}')
print(f'Artifact Location: {experiment.artifact_location}')
print(f'Tags: {experiment.tags}')
print(f'Lifecycle_stage: {experiment.lifecycle_stage}')
print(f'Creation timestamp: {experiment.creation_time}')

BATCH_SIZE = 128
SPLIT_SIZE = 0.8
RESCALES = [None, 1./255.]
ROTATION = None
TRANSLATION = None
BRIGHTEN = None
EPOCHS = 10


for RESCALE in RESCALES:
    with mlflow.start_run(
        run_name=f'run_rescale_{str(RESCALE)}',
        experiment_id=experiment_id
        ):
        mlflow.log_param('SPLIT_SIZE', SPLIT_SIZE)
        mlflow.log_param("BATCH_SIZE", BATCH_SIZE)
        mlflow.log_param('RESCALE', RESCALE)
        mlflow.log_param('ROTATION', ROTATION)
        mlflow.log_param('TRANSLATION', TRANSLATION)
        mlflow.log_param('BRIGHTEN', BRIGHTEN)
        mlflow.log_param('EPOCHS', EPOCHS)


        train_ds, val_ds = tf.keras.utils.split_dataset(
            data_loader.load_training_data(),
            left_size = SPLIT_SIZE,
            shuffle=True
        )

        train_ds = prepare(
            train_ds,
            rescale=RESCALE,
            batch_size=BATCH_SIZE,
            rotation=ROTATION,
            translation=TRANSLATION,
            brightness=BRIGHTEN
        )

        val_ds = prepare(
            val_ds,
            rescale=RESCALE,
            batch_size=BATCH_SIZE,
            rotation=ROTATION,
            translation=TRANSLATION,
            brightness=BRIGHTEN
        )
        
        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10)
            ])  
        model.compile(
            optimizer=tf.keras.optimizers.Adam(0.001),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['sparse_categorical_accuracy'])
        history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)


Name: pipeline-experiments-rescale_size
Experiment_id: 3
Artifact Location: file:///Users/b7064522/Documents/DataScience/kaggle-digit-recognizer/notebooks/../artifacts
Tags: {'version': 'v1', 'priority': 'P1'}
Lifecycle_stage: active
Creation timestamp: 1669994899543
Epoch 1/10
  1/263 [..............................] - ETA: 8:53 - loss: 144.6487 - sparse_categorical_accuracy: 0.1250WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0053s vs `on_train_batch_end` time: 0.0460s). Check your callbacks.


263/263 [==============================] - 5s 10ms/step - loss: 7.0660 - sparse_categorical_accuracy: 0.8351 - val_loss: 1.6385 - val_sparse_categorical_accuracy: 0.9030
Epoch 2/10
263/263 [==============================] - 2s 6ms/step - loss: 1.0925 - sparse_categorical_accuracy: 0.9085 - val_loss: 1.0314 - val_sparse_categorical_accuracy: 0.9019
Epoch 3/10
263/263 [==============================] - 1s 5ms/step - loss: 0.5351 - sparse_categorical_accuracy: 0.9305 - val_loss: 0.7939 - val_sparse_categorical_accuracy: 0.9126
Epoch 4/10
263/263 [==============================] - 1s 5ms/step - loss: 0.3543 - sparse_categorical_accuracy: 0.9425 - val_loss: 0.7705 - val_sparse_categorical_accuracy: 0.9238
Epoch 5/10
263/263 [==============================] - 1s 4ms/step - loss: 0.2567 - sparse_categorical_accuracy: 0.9524 - val_loss: 0.7230 - val_sparse_categorical_accuracy: 0.9211
Epoch 6/10
263/263 [==============================] - 1s 4ms/step - loss: 0.2082 - sparse_categorical_accuracy

2022/12/02 15:28:51 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/b7064522/opt/miniconda3/envs/kdrenv/lib/python3.10/site-packages/mlflow/tensorflow/__init__.py:189: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input."


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmp0739j1jo/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmp0739j1jo/model/data/model/assets


...

Epoch 1/10
  1/263 [..............................] - ETA: 8:42 - loss: 2.3549 - sparse_categorical_accuracy: 0.0938WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0047s vs `on_train_batch_end` time: 0.0567s). Check your callbacks.


263/263 [==============================] - 5s 12ms/step - loss: 0.4517 - sparse_categorical_accuracy: 0.8785 - val_loss: 0.2482 - val_sparse_categorical_accuracy: 0.9332
Epoch 2/10
263/263 [==============================] - 3s 12ms/step - loss: 0.2021 - sparse_categorical_accuracy: 0.9419 - val_loss: 0.1866 - val_sparse_categorical_accuracy: 0.9485
Epoch 3/10
263/263 [==============================] - 3s 10ms/step - loss: 0.1475 - sparse_categorical_accuracy: 0.9576 - val_loss: 0.1571 - val_sparse_categorical_accuracy: 0.9557
Epoch 4/10
263/263 [==============================] - 3s 9ms/step - loss: 0.1153 - sparse_categorical_accuracy: 0.9673 - val_loss: 0.1399 - val_sparse_categorical_accuracy: 0.9606
Epoch 5/10
263/263 [==============================] - 2s 8ms/step - loss: 0.0933 - sparse_categorical_accuracy: 0.9742 - val_loss: 0.1289 - val_sparse_categorical_accuracy: 0.9638
Epoch 6/10
263/263 [==============================] - 2s 8ms/step - loss: 0.0769 - sparse_categorical_accura

2022/12/02 15:29:52 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/b7064522/opt/miniconda3/envs/kdrenv/lib/python3.10/site-packages/mlflow/tensorflow/__init__.py:189: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input."


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmpibqbbhr4/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmpibqbbhr4/model/data/model/assets


## Experiment - add random rotation to the training set.

In [10]:
experiment_id = mlflow.create_experiment(
    'pipeline-experiments-rotation', #type: ignore
    artifact_location=Path.cwd().joinpath('../artifacts').as_uri(),
    tags={'version': 'v1', 'priority': 'P1'},
)
experiment = mlflow.get_experiment(experiment_id)
print(f'Name: {experiment.name}')
print(f'Experiment_id: {experiment.experiment_id}')
print(f'Artifact Location: {experiment.artifact_location}')
print(f'Tags: {experiment.tags}')
print(f'Lifecycle_stage: {experiment.lifecycle_stage}')
print(f'Creation timestamp: {experiment.creation_time}')

BATCH_SIZE = 128
SPLIT_SIZE = 0.8
RESCALE = 1./255
ROTATIONS = [0.05, 0.1, 0.15, 0.2, 0.25]
TRANSLATION = None
BRIGHTEN = None
EPOCHS = 10


for ROTATION in ROTATIONS:
    with mlflow.start_run(
        run_name=f'run_rotation_{str(ROTATION)}',
        experiment_id=experiment_id
        ):
        mlflow.log_param('SPLIT_SIZE', SPLIT_SIZE)
        mlflow.log_param("BATCH_SIZE", BATCH_SIZE)
        mlflow.log_param('RESCALE', RESCALE)
        mlflow.log_param('ROTATION', ROTATION)
        mlflow.log_param('TRANSLATION', TRANSLATION)
        mlflow.log_param('BRIGHTEN', BRIGHTEN)
        mlflow.log_param('EPOCHS', EPOCHS)


        train_ds, val_ds = tf.keras.utils.split_dataset(
            data_loader.load_training_data(),
            left_size = SPLIT_SIZE,
            shuffle=True
        )

        train_ds = prepare(
            train_ds,
            rescale=RESCALE,
            batch_size=BATCH_SIZE,
            rotation=ROTATION,
            translation=TRANSLATION,
            brightness=BRIGHTEN
        )

        val_ds = prepare(
            val_ds,
            rescale=RESCALE,
            batch_size=BATCH_SIZE,
            rotation=None,
            translation=None,
            brightness=None
        )
        
        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10)
            ])  
        model.compile(
            optimizer=tf.keras.optimizers.Adam(0.001),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['sparse_categorical_accuracy'])
        history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)


Name: pipeline-experiments-rotation
Experiment_id: 4
Artifact Location: file:///Users/b7064522/Documents/DataScience/kaggle-digit-recognizer/notebooks/../artifacts
Tags: {'version': 'v1', 'priority': 'P1'}
Lifecycle_stage: active
Creation timestamp: 1669995314046
Epoch 1/10
  5/263 [..............................] - ETA: 3s - loss: 2.1469 - sparse_categorical_accuracy: 0.2250  WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0128s vs `on_train_batch_end` time: 0.0350s). Check your callbacks.


263/263 [==============================] - 7s 19ms/step - loss: 0.5281 - sparse_categorical_accuracy: 0.8542 - val_loss: 0.2659 - val_sparse_categorical_accuracy: 0.9256
Epoch 2/10
263/263 [==============================] - 5s 18ms/step - loss: 0.2693 - sparse_categorical_accuracy: 0.9224 - val_loss: 0.1956 - val_sparse_categorical_accuracy: 0.9426
Epoch 3/10
263/263 [==============================] - 4s 13ms/step - loss: 0.2073 - sparse_categorical_accuracy: 0.9409 - val_loss: 0.1649 - val_sparse_categorical_accuracy: 0.9502
Epoch 4/10
263/263 [==============================] - 3s 11ms/step - loss: 0.1697 - sparse_categorical_accuracy: 0.9499 - val_loss: 0.1429 - val_sparse_categorical_accuracy: 0.9577
Epoch 5/10
263/263 [==============================] - 4s 17ms/step - loss: 0.1470 - sparse_categorical_accuracy: 0.9574 - val_loss: 0.1285 - val_sparse_categorical_accuracy: 0.9638
Epoch 6/10
263/263 [==============================] - 4s 16ms/step - loss: 0.1323 - sparse_categorical_acc

2022/12/02 15:36:07 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/b7064522/opt/miniconda3/envs/kdrenv/lib/python3.10/site-packages/mlflow/tensorflow/__init__.py:189: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input."


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmp76156n6b/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmp76156n6b/model/data/model/assets


...

Epoch 1/10
  6/263 [..............................] - ETA: 3s - loss: 2.2483 - sparse_categorical_accuracy: 0.1589  WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0115s vs `on_train_batch_end` time: 0.1317s). Check your callbacks.


263/263 [==============================] - 7s 20ms/step - loss: 0.9627 - sparse_categorical_accuracy: 0.7088 - val_loss: 0.5646 - val_sparse_categorical_accuracy: 0.8430
Epoch 2/10
263/263 [==============================] - 4s 15ms/step - loss: 0.5511 - sparse_categorical_accuracy: 0.8403 - val_loss: 0.4028 - val_sparse_categorical_accuracy: 0.8902
Epoch 3/10
263/263 [==============================] - 3s 12ms/step - loss: 0.4390 - sparse_categorical_accuracy: 0.8707 - val_loss: 0.3706 - val_sparse_categorical_accuracy: 0.8900
Epoch 4/10
263/263 [==============================] - 4s 14ms/step - loss: 0.3811 - sparse_categorical_accuracy: 0.8870 - val_loss: 0.3311 - val_sparse_categorical_accuracy: 0.9033
Epoch 5/10
263/263 [==============================] - 4s 17ms/step - loss: 0.3474 - sparse_categorical_accuracy: 0.8962 - val_loss: 0.3011 - val_sparse_categorical_accuracy: 0.9107
Epoch 6/10
263/263 [==============================] - 4s 17ms/step - loss: 0.3182 - sparse_categorical_acc

2022/12/02 15:40:26 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/b7064522/opt/miniconda3/envs/kdrenv/lib/python3.10/site-packages/mlflow/tensorflow/__init__.py:189: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input."


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmph36u5n89/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmph36u5n89/model/data/model/assets


## Experiment - random translations

In [14]:
experiment_id = mlflow.create_experiment(
    'pipeline-experiments-translation-rerun', #type: ignore
    artifact_location=Path.cwd().joinpath('../artifacts').as_uri(),
    tags={'version': 'v1', 'priority': 'P1'},
)
experiment = mlflow.get_experiment(experiment_id)
print(f'Name: {experiment.name}')
print(f'Experiment_id: {experiment.experiment_id}')
print(f'Artifact Location: {experiment.artifact_location}')
print(f'Tags: {experiment.tags}')
print(f'Lifecycle_stage: {experiment.lifecycle_stage}')
print(f'Creation timestamp: {experiment.creation_time}')

BATCH_SIZE = 128
SPLIT_SIZE = 0.8
RESCALE = 1./255
ROTATION = 0.05
TRANSLATIONS = [(0.025, 0.025), (0.05,0.05), (0.1,0.1), (0.2, 0.2), (0.0, 0.1), (0.1, 0.0)]
BRIGHTEN = None
EPOCHS = 10


for TRANSLATION in TRANSLATIONS:
    with mlflow.start_run(
        run_name=f'run_translation_{str(TRANSLATION[0])}_{str(TRANSLATION[1])}',
        experiment_id=experiment_id
        ):
        mlflow.log_param('SPLIT_SIZE', SPLIT_SIZE)
        mlflow.log_param("BATCH_SIZE", BATCH_SIZE)
        mlflow.log_param('RESCALE', RESCALE)
        mlflow.log_param('ROTATION', ROTATION)
        mlflow.log_param('TRANSLATION', TRANSLATION)
        mlflow.log_param('BRIGHTEN', BRIGHTEN)
        mlflow.log_param('EPOCHS', EPOCHS)


        train_ds, val_ds = tf.keras.utils.split_dataset(
            data_loader.load_training_data(),
            left_size = SPLIT_SIZE,
            shuffle=True
        )

        train_ds = prepare(
            train_ds,
            rescale=RESCALE,
            batch_size=BATCH_SIZE,
            rotation=ROTATION,
            translation=TRANSLATION,
            brightness=BRIGHTEN
        )

        val_ds = prepare(
            val_ds,
            rescale=RESCALE,
            batch_size=BATCH_SIZE,
            rotation=None,
            translation=None,
            brightness=None
        )
        
        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10)
            ])  
        model.compile(
            optimizer=tf.keras.optimizers.Adam(0.001),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['sparse_categorical_accuracy'])
        history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)


Name: pipeline-experiments-translation-rerun
Experiment_id: 6
Artifact Location: file:///Users/b7064522/Documents/DataScience/kaggle-digit-recognizer/notebooks/../artifacts
Tags: {'version': 'v1', 'priority': 'P1'}
Lifecycle_stage: active
Creation timestamp: 1669997123916
Epoch 1/10
  5/263 [..............................] - ETA: 7s - loss: 2.1449 - sparse_categorical_accuracy: 0.2641 WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0240s vs `on_train_batch_end` time: 0.0339s). Check your callbacks.


263/263 [==============================] - 10s 33ms/step - loss: 0.5783 - sparse_categorical_accuracy: 0.8395 - val_loss: 0.2672 - val_sparse_categorical_accuracy: 0.9224
Epoch 2/10
263/263 [==============================] - 6s 24ms/step - loss: 0.3150 - sparse_categorical_accuracy: 0.9086 - val_loss: 0.1971 - val_sparse_categorical_accuracy: 0.9413
Epoch 3/10
263/263 [==============================] - 5s 20ms/step - loss: 0.2418 - sparse_categorical_accuracy: 0.9293 - val_loss: 0.1566 - val_sparse_categorical_accuracy: 0.9529
Epoch 4/10
263/263 [==============================] - 5s 21ms/step - loss: 0.2049 - sparse_categorical_accuracy: 0.9406 - val_loss: 0.1339 - val_sparse_categorical_accuracy: 0.9598
Epoch 5/10
263/263 [==============================] - 5s 20ms/step - loss: 0.1744 - sparse_categorical_accuracy: 0.9499 - val_loss: 0.1201 - val_sparse_categorical_accuracy: 0.9644
Epoch 6/10
263/263 [==============================] - 6s 21ms/step - loss: 0.1561 - sparse_categorical_ac

2022/12/02 16:06:38 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/b7064522/opt/miniconda3/envs/kdrenv/lib/python3.10/site-packages/mlflow/tensorflow/__init__.py:189: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input."


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmp4eapmpce/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmp4eapmpce/model/data/model/assets


...

Epoch 1/10
  5/263 [..............................] - ETA: 12s - loss: 2.2374 - sparse_categorical_accuracy: 0.1547WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0238s vs `on_train_batch_end` time: 0.0370s). Check your callbacks.


263/263 [==============================] - 11s 34ms/step - loss: 0.9780 - sparse_categorical_accuracy: 0.7045 - val_loss: 0.4275 - val_sparse_categorical_accuracy: 0.8908
Epoch 2/10
263/263 [==============================] - 6s 22ms/step - loss: 0.5098 - sparse_categorical_accuracy: 0.8555 - val_loss: 0.2797 - val_sparse_categorical_accuracy: 0.9287
Epoch 3/10
263/263 [==============================] - 6s 23ms/step - loss: 0.3650 - sparse_categorical_accuracy: 0.8968 - val_loss: 0.2119 - val_sparse_categorical_accuracy: 0.9463
Epoch 4/10
263/263 [==============================] - 7s 25ms/step - loss: 0.2999 - sparse_categorical_accuracy: 0.9131 - val_loss: 0.1824 - val_sparse_categorical_accuracy: 0.9510
Epoch 5/10
263/263 [==============================] - 7s 25ms/step - loss: 0.2643 - sparse_categorical_accuracy: 0.9227 - val_loss: 0.1794 - val_sparse_categorical_accuracy: 0.9519
Epoch 6/10
263/263 [==============================] - 9s 34ms/step - loss: 0.2378 - sparse_categorical_ac

2022/12/02 16:16:53 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/b7064522/opt/miniconda3/envs/kdrenv/lib/python3.10/site-packages/mlflow/tensorflow/__init__.py:189: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input."


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmpgzhllvnh/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmpgzhllvnh/model/data/model/assets


## Experiment - random brightness

In [15]:
experiment_id = mlflow.create_experiment(
    'pipeline-experiments-brightness', #type: ignore
    artifact_location=Path.cwd().joinpath('../artifacts').as_uri(),
    tags={'version': 'v1', 'priority': 'P1'},
)
experiment = mlflow.get_experiment(experiment_id)
print(f'Name: {experiment.name}')
print(f'Experiment_id: {experiment.experiment_id}')
print(f'Artifact Location: {experiment.artifact_location}')
print(f'Tags: {experiment.tags}')
print(f'Lifecycle_stage: {experiment.lifecycle_stage}')
print(f'Creation timestamp: {experiment.creation_time}')

BATCH_SIZE = 128
SPLIT_SIZE = 0.8
RESCALE = 1./255
ROTATION = 0.05
TRANSLATION = (0.1, 0.1)
BRIGHTENS = [0.05, 0.1, 0.2, 0.3]
EPOCHS = 10


for BRIGHTEN in BRIGHTENS:
    with mlflow.start_run(
        run_name=f'run_brighten_{str(BRIGHTEN)}',
        experiment_id=experiment_id
        ):
        mlflow.log_param('SPLIT_SIZE', SPLIT_SIZE)
        mlflow.log_param("BATCH_SIZE", BATCH_SIZE)
        mlflow.log_param('RESCALE', RESCALE)
        mlflow.log_param('ROTATION', ROTATION)
        mlflow.log_param('TRANSLATION', TRANSLATION)
        mlflow.log_param('BRIGHTEN', BRIGHTEN)
        mlflow.log_param('EPOCHS', EPOCHS)


        train_ds, val_ds = tf.keras.utils.split_dataset(
            data_loader.load_training_data(),
            left_size = SPLIT_SIZE,
            shuffle=True
        )

        train_ds = prepare(
            train_ds,
            rescale=RESCALE,
            batch_size=BATCH_SIZE,
            rotation=ROTATION,
            translation=TRANSLATION,
            brightness=BRIGHTEN
        )

        val_ds = prepare(
            val_ds,
            rescale=RESCALE,
            batch_size=BATCH_SIZE,
            rotation=None,
            translation=None,
            brightness=None
        )
        
        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10)
            ])  
        model.compile(
            optimizer=tf.keras.optimizers.Adam(0.001),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['sparse_categorical_accuracy'])
        history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)


Name: pipeline-experiments-brightness
Experiment_id: 7
Artifact Location: file:///Users/b7064522/Documents/DataScience/kaggle-digit-recognizer/notebooks/../artifacts
Tags: {'version': 'v1', 'priority': 'P1'}
Lifecycle_stage: active
Creation timestamp: 1669998180228
Epoch 1/10
  4/263 [..............................] - ETA: 5s - loss: 2.2885 - sparse_categorical_accuracy: 0.1562  WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0188s vs `on_train_batch_end` time: 0.0332s). Check your callbacks.


263/263 [==============================] - 12s 39ms/step - loss: 1.0064 - sparse_categorical_accuracy: 0.6942 - val_loss: 0.4371 - val_sparse_categorical_accuracy: 0.8902
Epoch 2/10
263/263 [==============================] - 8s 32ms/step - loss: 0.5243 - sparse_categorical_accuracy: 0.8502 - val_loss: 0.2754 - val_sparse_categorical_accuracy: 0.9288
Epoch 3/10
263/263 [==============================] - 9s 34ms/step - loss: 0.3792 - sparse_categorical_accuracy: 0.8921 - val_loss: 0.2106 - val_sparse_categorical_accuracy: 0.9442
Epoch 4/10
263/263 [==============================] - 9s 33ms/step - loss: 0.3095 - sparse_categorical_accuracy: 0.9092 - val_loss: 0.1858 - val_sparse_categorical_accuracy: 0.9495
Epoch 5/10
263/263 [==============================] - 8s 28ms/step - loss: 0.2731 - sparse_categorical_accuracy: 0.9203 - val_loss: 0.1651 - val_sparse_categorical_accuracy: 0.9530
Epoch 6/10
263/263 [==============================] - 6s 21ms/step - loss: 0.2503 - sparse_categorical_ac

2022/12/02 16:24:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/b7064522/opt/miniconda3/envs/kdrenv/lib/python3.10/site-packages/mlflow/tensorflow/__init__.py:189: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input."


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmpexwa4hot/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmpexwa4hot/model/data/model/assets


...

Epoch 1/10
  5/263 [..............................] - ETA: 7s - loss: 2.3260 - sparse_categorical_accuracy: 0.0984 WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0232s vs `on_train_batch_end` time: 0.0529s). Check your callbacks.


263/263 [==============================] - 13s 41ms/step - loss: 1.2012 - sparse_categorical_accuracy: 0.6265 - val_loss: 0.4461 - val_sparse_categorical_accuracy: 0.8874
Epoch 2/10
263/263 [==============================] - 12s 44ms/step - loss: 0.6740 - sparse_categorical_accuracy: 0.8087 - val_loss: 0.2845 - val_sparse_categorical_accuracy: 0.9301
Epoch 3/10
263/263 [==============================] - 9s 34ms/step - loss: 0.4861 - sparse_categorical_accuracy: 0.8605 - val_loss: 0.2102 - val_sparse_categorical_accuracy: 0.9433
Epoch 4/10
263/263 [==============================] - 7s 26ms/step - loss: 0.3911 - sparse_categorical_accuracy: 0.8874 - val_loss: 0.1725 - val_sparse_categorical_accuracy: 0.9535
Epoch 5/10
263/263 [==============================] - 6s 22ms/step - loss: 0.3432 - sparse_categorical_accuracy: 0.9030 - val_loss: 0.1548 - val_sparse_categorical_accuracy: 0.9558
Epoch 6/10
263/263 [==============================] - 5s 18ms/step - loss: 0.3107 - sparse_categorical_a

2022/12/02 16:31:25 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/b7064522/opt/miniconda3/envs/kdrenv/lib/python3.10/site-packages/mlflow/tensorflow/__init__.py:189: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input."


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmpihfzcwue/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmpihfzcwue/model/data/model/assets


## Experiment Random zoom

In [18]:
experiment_id = mlflow.create_experiment(
    'pipeline-experiments-zoom_rerun', #type: ignore
    artifact_location=Path.cwd().joinpath('../artifacts').as_uri(),
    tags={'version': 'v1', 'priority': 'P1'},
)
experiment = mlflow.get_experiment(experiment_id)
print(f'Name: {experiment.name}')
print(f'Experiment_id: {experiment.experiment_id}')
print(f'Artifact Location: {experiment.artifact_location}')
print(f'Tags: {experiment.tags}')
print(f'Lifecycle_stage: {experiment.lifecycle_stage}')
print(f'Creation timestamp: {experiment.creation_time}')

BATCH_SIZE = 128
SPLIT_SIZE = 0.8
RESCALE = 1./255
ROTATION = 0.05
TRANSLATION = (0.1, 0.1)
BRIGHTENS = 0.1
ZOOMS = [0.05, 0.1, 0.2]
EPOCHS = 10


for ZOOM in ZOOMS:
    with mlflow.start_run(
        run_name=f'run_zoom_{str(ZOOM)}',
        experiment_id=experiment_id
        ):
        mlflow.log_param('SPLIT_SIZE', SPLIT_SIZE)
        mlflow.log_param("BATCH_SIZE", BATCH_SIZE)
        mlflow.log_param('RESCALE', RESCALE)
        mlflow.log_param('ROTATION', ROTATION)
        mlflow.log_param('TRANSLATION', TRANSLATION)
        mlflow.log_param('BRIGHTEN', BRIGHTEN)
        mlflow.log_param('ZOOM', ZOOM)
        mlflow.log_param('EPOCHS', EPOCHS)


        train_ds, val_ds = tf.keras.utils.split_dataset(
            data_loader.load_training_data(),
            left_size = SPLIT_SIZE,
            shuffle=True
        )

        train_ds = prepare(
            train_ds,
            rescale=RESCALE,
            batch_size=BATCH_SIZE,
            rotation=ROTATION,
            translation=TRANSLATION,
            brightness=BRIGHTEN,
            zoom=ZOOM
        )

        val_ds = prepare(
            val_ds,
            rescale=RESCALE,
            batch_size=BATCH_SIZE,
            rotation=None,
            translation=None,
            brightness=None,
            zoom=None
        )
        
        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10)
            ])  
        model.compile(
            optimizer=tf.keras.optimizers.Adam(0.001),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['sparse_categorical_accuracy'])
        history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)


Name: pipeline-experiments-zoom_rerun
Experiment_id: 9
Artifact Location: file:///Users/b7064522/Documents/DataScience/kaggle-digit-recognizer/notebooks/../artifacts
Tags: {'version': 'v1', 'priority': 'P1'}
Lifecycle_stage: active
Creation timestamp: 1669999343678
Epoch 1/10
  6/263 [..............................] - ETA: 10s - loss: 2.2425 - sparse_categorical_accuracy: 0.2161WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0354s vs `on_train_batch_end` time: 0.0579s). Check your callbacks.


263/263 [==============================] - 21s 72ms/step - loss: 1.1867 - sparse_categorical_accuracy: 0.6335 - val_loss: 0.4397 - val_sparse_categorical_accuracy: 0.8833
Epoch 2/10
263/263 [==============================] - 10s 39ms/step - loss: 0.6807 - sparse_categorical_accuracy: 0.8057 - val_loss: 0.2826 - val_sparse_categorical_accuracy: 0.9248
Epoch 3/10
263/263 [==============================] - 9s 32ms/step - loss: 0.4838 - sparse_categorical_accuracy: 0.8635 - val_loss: 0.2153 - val_sparse_categorical_accuracy: 0.9421
Epoch 4/10
263/263 [==============================] - 13s 49ms/step - loss: 0.3973 - sparse_categorical_accuracy: 0.8872 - val_loss: 0.1821 - val_sparse_categorical_accuracy: 0.9474
Epoch 5/10
263/263 [==============================] - 12s 45ms/step - loss: 0.3428 - sparse_categorical_accuracy: 0.9017 - val_loss: 0.1638 - val_sparse_categorical_accuracy: 0.9489
Epoch 6/10
263/263 [==============================] - 9s 35ms/step - loss: 0.3111 - sparse_categorical

2022/12/02 16:45:09 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/b7064522/opt/miniconda3/envs/kdrenv/lib/python3.10/site-packages/mlflow/tensorflow/__init__.py:189: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input."


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmp22pi46f_/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmp22pi46f_/model/data/model/assets


...

Epoch 1/10
  6/263 [..............................] - ETA: 14s - loss: 2.2842 - sparse_categorical_accuracy: 0.1771WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0334s vs `on_train_batch_end` time: 0.0497s). Check your callbacks.


263/263 [==============================] - 14s 46ms/step - loss: 1.2954 - sparse_categorical_accuracy: 0.5990 - val_loss: 0.4308 - val_sparse_categorical_accuracy: 0.8943
Epoch 2/10
263/263 [==============================] - 6s 24ms/step - loss: 0.7442 - sparse_categorical_accuracy: 0.7889 - val_loss: 0.2714 - val_sparse_categorical_accuracy: 0.9270
Epoch 3/10
263/263 [==============================] - 6s 24ms/step - loss: 0.5393 - sparse_categorical_accuracy: 0.8492 - val_loss: 0.2113 - val_sparse_categorical_accuracy: 0.9414
Epoch 4/10
263/263 [==============================] - 6s 24ms/step - loss: 0.4483 - sparse_categorical_accuracy: 0.8751 - val_loss: 0.1768 - val_sparse_categorical_accuracy: 0.9510
Epoch 5/10
263/263 [==============================] - 6s 24ms/step - loss: 0.3843 - sparse_categorical_accuracy: 0.8910 - val_loss: 0.1546 - val_sparse_categorical_accuracy: 0.9569
Epoch 6/10
263/263 [==============================] - 6s 24ms/step - loss: 0.3555 - sparse_categorical_ac

2022/12/02 16:49:55 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/b7064522/opt/miniconda3/envs/kdrenv/lib/python3.10/site-packages/mlflow/tensorflow/__init__.py:189: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input."


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmpk9tgkbg8/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/4g/6c1htv314w9dcb25vhpxqfscjwsh63/T/tmpk9tgkbg8/model/data/model/assets
